### 5.2 핵심 기능

Series나 DataFrame에 저장된 데이터를 다루는 기본 방법을 설명합니다.

#### 5.2.1 재색인

``reindex``

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [2]:
obj = Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [3]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'], fill_value=0)
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    0.0
dtype: float64

시계열 같은 순차적인 데이터를 재색인할 때는 값을 보간하거나 채워 넣어야 할 경우가 많습니다.  
이런 경우 ``method`` 옵션을 이용해서 해결할 수 있습니다.  
``ffill``

In [4]:
obj3 = Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

- reindx 메서드(보간) 옵션
    - ``ffill`` or ``pad``: 앞의 값으로 채워 넣습니다.
    - ``bfill`` or ``backfill``: 뒤의 값으로 채워 넣습니다.

In [5]:
frame = DataFrame(np.arange(9).reshape(3, 3), index=['a', 'c', 'd'],
                  columns=['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [6]:
# option 추가: method='ffill', fill_value=0
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


``columns``: 세로축의 reindexing 이 가능합니다.

In [7]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


In [8]:
frame.reindex(index=['a', 'b', 'c', 'd'], columns=states)

,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


row b에 보간을 ffill 메서드를 이용해서 해보자  
보간은 axis=0 즉, row에 대해서만 이루어집니다.

In [9]:
# frame.reindex(index=['a','b','c','d'], method='ffill', columns=states)

[오류 해결 stackoverflow 답](https://stackoverflow.com/questions/56948060/index-must-be-monotonic-increasing-or-decreasing)

책에 있는데로 하면 오류가 나와서 이렇게 수정했습니다.  
If your index and columns are strings and not monotonic, you need to call .ffill, .bfill or .fillna outside after of reindex

In [10]:
myframe = DataFrame(np.arange(9).reshape(3, 3), index=['a', 'c', 'd'],
                 columns=['Ohio', 'Texas', 'California'])
myframe.reindex(index=['a', 'b', 'c', 'd'], columns=['Texas', 'Utah', 'California']).ffill()

,Texas,Utah,California
a,1.0,NaN,2.0
b,1.0,NaN,2.0
c,4.0,NaN,5.0
d,7.0,NaN,8.0


재색인은 ``ix``를 이용해서 라벨로 색인하면 좀 더 간결할 수 있습니다.  
하지만 ``ix``를 사용하면 경고가 나오며  
``loc``(label based indexing) 또는 ``iloc``(positional indexing)를 사용하라고 합니다.  

[참고 문서](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated)

In [11]:
frame.loc[['a', 'b', 'c', 'd'], states]

C:\Users\wansang\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


재색인 함수 인자

- index: 색인으로 사용할 새로운 순서, index 인스턴스나 다른 순차적인 자료 구조를 사용할 수 있습니다.
- method: 보간시 사용
- fill_value: 재색인 과정 중에 새롭게 나타나는 비어있는 데이터를 채우기 위한 값
- limit: 전/후 보간 시에 사용할 최대 갭 크기
- level: MultiIndex 단계(level)에 단순 색인을 맞춥니다. 그렇지 않으면 MultiIndex의 하위 부분집합에 맞춥니다.
- copy: True인 경우 새로운 색인이 이전 색인과 같더라도 데이터를 복사합니다. False라면 두 색인이 같을 경우 데이터를 복사하지 않습니다.

#### 5.2.2 하나의 로우 또는 칼럼 삭제하기

``drop`` 메서드를 사용하면 선택한 값이 삭제된 새로운 객체를 얻을 수 있습니다.

In [12]:
obj = Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [13]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

In [14]:
data = DataFrame(np.arange(16).reshape((4, 4)),
                index=['Ohio', 'Colorado', 'Utah', 'New York'],
                columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [15]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [16]:
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [17]:
data.drop(['two', 'four'], axis=1)

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


#### 5.2.3 색인하기, 선택하기, 거르기

Series의 색인은 NumPy 배열의 색인과 유사하게 동작하는데, Series의 색인은 정수가 아니어도 된다는 점이 다릅니다.

In [18]:
obj = Series(np.arange(4.), index=['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [19]:
print(obj['b'])  # 'b'의 값
print(obj[1])  # 1번째

1.0
1.0


In [20]:
print(obj[2:4])
print(obj[['b', 'a', 'd']])
print(obj[[1, 3]])
print(obj[obj < 2])

c    2.0
d    3.0
dtype: float64
b    1.0
a    0.0
d    3.0
dtype: float64
b    1.0
d    3.0
dtype: float64
a    0.0
b    1.0
dtype: float64


In [21]:
obj['b':'d']  # 슬라이싱은 파이썬과 다르게 끝값을 포함합니다.

b    1.0
c    2.0
d    3.0
dtype: float64

In [22]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

DataFrame에서 칼럼의 값을 하나 이상 가져올 수 있습니다.

In [23]:
data = DataFrame(np.arange(16).reshape((4, 4)),
                index=['Ohio', 'Colorado', 'Utah', 'New York'],
                columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [24]:
data['two'][0:3]

Ohio        1
Colorado    5
Utah        9
Name: two, dtype: int32

In [25]:
data[['two', 'three']][0:3]

,two,three
Ohio,1,2
Colorado,5,6
Utah,9,10


In [27]:
data[data['three'] > 9]

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [28]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [30]:
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [31]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int32

In [34]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


.ix 대신에 .loc, .iloc 사용하는 방법 예제

In [85]:
dfd = pd.DataFrame({'A': [1, 2, 3],
                    'B': [4, 5, 6]},
                   index=list('abc'))
print(dfd)
print('==========')
print(dfd.loc[dfd.index[[0, 2]], 'A'])
print(dfd.iloc[[0, 2], dfd.columns.get_loc('A')])
print(dfd.iloc[[0, 2], dfd.columns.get_indexer(['A', 'B'])])

   A  B
a  1  4
b  2  5
c  3  6
a    1
c    3
Name: A, dtype: int64
a    1
c    3
Name: A, dtype: int64
   A  B
a  1  4
c  3  6


In [86]:
# data.ix[['Colorado', 'Utah'], [3, 0, 1]]
data.iloc[data.index.get_indexer(['Colorado', 'Utah']), [3, 0, 1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


In [88]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

In [90]:
data.loc[:'Utah', :'two']

,one,two
Ohio,0,0
Colorado,0,5
Utah,8,9


In [111]:
# data.ix[data.three > 5, :3]
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


#### 5.2.4 산술연산과 데이터 정렬

판다스는 객체간 산술연산이 가능합니다.  
짝이 맞지 않는 index가 있다면 두 index는 통합됩니다.  

In [115]:
s1 = Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = Series([-2.1, 3.6, -1.5, 4, 3.1], index=['a', 'c', 'e', 'f', 'g'])
print(s1)
print(s2)

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64
a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64


In [114]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

DataFrame에서는 로우와 칼럼 모두 적용됩니다.

In [117]:
df1 = DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
               index=['Ohio', 'Texas', 'Colorado'])
df2 = DataFrame(np.arange(12).reshape((4, 3)), columns=list('bde'),
               index=['Utah', 'Ohio', 'Texas', 'Oregon'])
print(df1)
print(df2)

            b    c    d
Ohio      0.0  1.0  2.0
Texas     3.0  4.0  5.0
Colorado  6.0  7.0  8.0
        b   d   e
Utah    0   1   2
Ohio    3   4   5
Texas   6   7   8
Oregon  9  10  11


In [120]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


산술연산 메서드에 채워 넣을 값 지정하기

In [130]:
df1 = DataFrame(np.arange(12.).reshape((3, 4)), columns=list('abcd'))
df2 = DataFrame(np.arange(20.).reshape((4, 5)), columns=list('abcde'))
print(df1, df2, sep='\n')

     a    b     c     d
0  0.0  1.0   2.0   3.0
1  4.0  5.0   6.0   7.0
2  8.0  9.0  10.0  11.0
      a     b     c     d     e
0   0.0   1.0   2.0   3.0   4.0
1   5.0   6.0   7.0   8.0   9.0
2  10.0  11.0  12.0  13.0  14.0
3  15.0  16.0  17.0  18.0  19.0


In [133]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,11.0,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [138]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,11.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


In [136]:
df1.reindex(index=df2.index, columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0.0
1,4.0,5.0,6.0,7.0,0.0
2,8.0,9.0,10.0,11.0,0.0
3,0.0,0.0,0.0,0.0,0.0


DataFrame과 Series 간의 연산

In [140]:
arr = np.arange(12.).reshape((3, 4))
arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [141]:
arr[0]

array([0., 1., 2., 3.])

In [142]:
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

In [144]:
frame = DataFrame(np.arange(12).reshape((4, 3)), columns=list('bde'),
                 index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0,1,2
Ohio,3,4,5
Texas,6,7,8
Oregon,9,10,11


In [146]:
series = frame.iloc[0]
series

b    0
d    1
e    2
Name: Utah, dtype: int32

In [147]:
frame - series

,b,d,e
Utah,0,0,0
Ohio,3,3,3
Texas,6,6,6
Oregon,9,9,9


In [148]:
series2 = Series(range(3), index=['b', 'e', 'f'])
series2

b    0
e    1
f    2
dtype: int64

In [152]:
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


In [153]:
series3 = frame['d']
series3

Utah       1
Ohio       4
Texas      7
Oregon    10
Name: d, dtype: int32

In [154]:
frame.sub(series3, axis=0)

,b,d,e
Utah,-1,0,1
Ohio,-1,0,1
Texas,-1,0,1
Oregon,-1,0,1


#### 5.2.5 함수 적용과 매핑

판다스 객체에도 넘파이의 유니버설 함수를 적용할 수 있습니다.

In [191]:
frame = DataFrame(np.random.randn(4, 3), columns=list('bde'),
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,-0.834964,-0.936281,0.982842
Ohio,-0.556738,1.604816,-1.431748
Texas,-0.048292,-1.548317,0.679583
Oregon,-0.078725,-0.355610,0.905813


In [192]:
np.abs(frame)

,b,d,e
Utah,0.834964,0.936281,0.982842
Ohio,0.556738,1.604816,1.431748
Texas,0.048292,1.548317,0.679583
Oregon,0.078725,0.355610,0.905813


In [193]:
f = lambda x: x.max() - x.min()
frame.apply(f)

b    0.786672
d    3.153134
e    2.414590
dtype: float64

In [194]:
frame.apply(f, axis=1)

Utah      1.919122
Ohio      3.036564
Texas     2.227900
Oregon    1.261423
dtype: float64

In [195]:
def f(x):
    return Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

,b,d,e
min,-0.834964,-1.548317,-1.431748
max,-0.048292,1.604816,0.982842


각 원소에 적용되는 파이썬 함수를 사용하고 싶으면 ``applymap``을 사용하면 됩니다.  
이 함수의 이름이 map이 붙는 이유는 Series가 각 원소에 적용할 함수를 지정하기 위한 map 메서드를 가지고 있기 때문입니다.

In [196]:
format = lambda x: f'{x:.2f}'
frame.applymap(format)

,b,d,e
Utah,-0.83,-0.94,0.98
Ohio,-0.56,1.60,-1.43
Texas,-0.05,-1.55,0.68
Oregon,-0.08,-0.36,0.91


In [197]:
frame['e'].map(format)

Utah       0.98
Ohio      -1.43
Texas      0.68
Oregon     0.91
Name: e, dtype: object

#### 5.2.6 정렬과 순위

데이터를 정렬해보자

In [198]:
obj = Series(range(4), index=['d', 'a', 'b', 'c'])
obj

d    0
a    1
b    2
c    3
dtype: int64

In [199]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

In [200]:
obj.sort_values()

d    0
a    1
b    2
c    3
dtype: int64

DataFrame은 로우나 칼럼 중 하나의 축을 기준으로 정렬 할 수 있습니다.

In [211]:
frame = DataFrame(np.arange(8).reshape((2, 4)), index=['three', 'one'],
                 columns=['d', 'a', 'b', 'c'])
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [212]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [215]:
frame.sort_index(axis=1, ascending=False)  # 축, 내림차순 설정

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


In [217]:
obj = Series([4, 7, -3, 2])
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

정렬할 때 비어있는 값은 기본적으로 마지막에 위치합니다.

In [219]:
obj = Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

DataFrame에서는 하나의 이상의 칼럼에 있는 값으로 정렬이 필요할 수 있습니다.  
이럴 때는 by 옵션에 필요한 칼럼의 이름을 넘기면 됩니다.

In [221]:
frame = DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})
frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


In [227]:
frame.sort_values(by='b', ascending=True)

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


In [229]:
# 정렬 우선순위 정하기
frame.sort_values(by=['a', 'b'])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


In [232]:
obj = Series([7, -5, 7, 4, 2, 0, 4, 8, 8, 8])
obj.rank()
# 0번째 6.5등(0번째와 2번째가 동률이라 6등, 7등의 평균으로 계산)
# 7, 8, 9번째 9등(8, 9, 10등이 동률이라 평균으로 계산)

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
7    9.0
8    9.0
9    9.0
dtype: float64

In [234]:
obj.rank(method='first')  # 데이터 상의 나타나는 순위를 매길 수도 있습니다.

0     6.0
1     1.0
2     7.0
3     4.0
4     3.0
5     2.0
6     5.0
7     8.0
8     9.0
9    10.0
dtype: float64

In [237]:
obj.rank(ascending=False, method='max')  # 내림차순으로 순위를 매김

0     5.0
1    10.0
2     5.0
3     7.0
4     8.0
5     9.0
6     7.0
7     3.0
8     3.0
9     3.0
dtype: float64

In [238]:
frame = DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0 , 1],
                  'c': [-2, 5, 8, -2.5]})
frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [240]:
frame.rank(axis=1, ascending=False, method='first')

,b,a,c
0,1.0,2.0,3.0
1,1.0,3.0,2.0
2,3.0,2.0,1.0
3,1.0,2.0,3.0


rank 함수의 동률을 처리하는 메서드

- 'average': 기본 값: 같은 값을 가지는 항목의 평균 값을 순위로 삼습니다.
- 'min': 같은 값을 가지는 그룹을 낮은 순위로 매깁니다.
- 'max': 같은 값을 가지는 그룹을 높은 순위로 매깁니다.
- 'first': 데이터 내에서 위치에 따라 순위를 매깁니다.

#### 5.2.7 중복 색인

index가 중복인 경우를 살펴보자

In [243]:
obj = Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

index의 ``is_unique`` 속성은 해당 값이 유일한지 아닌지 알려줍니다.

In [245]:
obj.index.is_unique

False

중복된 index는 Series 객체로 반환되고 중복되지 않은 index는 스칼라 값을 반환합니다.

In [248]:
print(obj['a'])
print(obj['c'])

a    0
a    1
dtype: int64
4


In [251]:
df = DataFrame(np.random.randn(4, 3), index=['a', 'a', 'b', 'b'])
df

,0,1,2
a,0.311730,-0.229584,1.039989
a,-1.237163,0.908865,0.013292
b,-1.867748,-0.113197,0.411322
b,-0.015819,0.964010,-0.629854


In [253]:
df.loc['b']

,0,1,2
b,-1.867748,-0.113197,0.411322
b,-0.015819,0.964010,-0.629854
